In [1]:
import sys
sys.path.append('../../lib/python3.5/site-packages')
import re
import jieba.analyse
import numpy as np
import jieba
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words("./ch_dont_want_word")

In [2]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

es = Elasticsearch([{'host': '192.168.2.10', 'port': 9200}])

In [3]:
# es.get(index="pixnet", doc_type='food', id=2)

search_body = \
{\
  "_source": ["title","content","tags"], \
  "query" : {\
    "bool": {\
      "must": [\
        { \
          "match" : { \
            "content" : "台中"\
          }\
        },\
        { \
          "match" : { \
            "content" : "早午餐"\
          }\
        },\
#         { \
#           "match" : { \
#             "tags" : "夜"\
#           }\
#         }\
      ],\
#       "filter": \
#         [\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "新竹"\
#               }\
#             }\
#           },\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "宵夜"\
#               }\
#             }\
#           }\
#         ]\
    }\
  }\
}
# Take top 10
res = es.search(index="pixnet", doc_type="food", body= search_body)
print("%d documents found" % res['hits']['total'])
for i, match in enumerate(res['hits']['hits']):
    print("Top.{} Match\nScore : {}\nTitle : {}\nContent : {}\n\n".format(i, match['_score'], match['_source']['title'], match['_source']['content']))
    break

1094 documents found
Top.0 Match
Score : 14.739376
Title : 【台中早午餐鬆餅】【田樂】老宅新生~~輕鬆吃田樂小公園店!
Content :  
｜店家資訊｜
田樂●小公園店  (for Farm Burger/田樂二店)
地址：台中市西區大忠南街82巷36號
官方網站 ：http://forfarmburger.blogspot.tw/
電話：04-23728672
 因為Pull沒辦法一一追蹤店家的現況喔！
建議大家可以先打電話確認一下^0^
 
話說上次本來要吃田樂一店的貪吃母女檔沒有吃到田樂
反而發現祕密廚房開分店了~就是...《小祕密廚房》
然後就在那兒度過了美好的中午!
不過回到家後...Pull媽還是心繫田樂
一直對著Pull轟炸說她要吃田樂她要吃田樂!!!!!
於是Pull想說一定要找一天和Pull媽去田樂不然她會哭哭~~~
剛好某天體驗完胖達之後
肚子還有點空虛...
(不是真的餓~是心靈上的空虛啦)
結果腦中突然想到了田樂!
於是我們倆就決議拜訪田樂小公園店(田樂二店)
(因為上次雖然沒吃到但有看到一店的用餐環境...好擁擠)
於是就開啟了『Pull和Pull媽美食吃不停之田樂二店的奇幻旅程』
 

田樂小公園店(以下簡稱田樂)位在大忠南街82巷
因為在很隱密的巷弄內
所以要很仔細地尋找喔!不然會忽略掉...
走進82巷後再往裡面一直走
就會發現一個比較老舊的社區
田樂就在最顯眼的那個角角了!
Pull覺得最近老宅改建/翻修的案例真的很多耶
而且也都成功製造了話題喔
像是田樂最近在網路上可說是紅翻天啊
於是乎也造成了上次我們去田樂一店卻吃不到的悲劇
這次其實我們也不抱很多希望...
因為一店就滿成這樣了
那二店豈不是更恐怖~~~
只是想說碰個運氣看看會不會剛好被我們訂到位!
  
 田樂的外觀就是老宅囉!
這個木牌蠻酷的欸
真的有很舊很舊的感覺~好藝術!
  
 外面有種花花草草...真是賞心悅目!
窗戶透露著黃光的溫暖氣息
  
 總是免不了要放小立板在外頭!
田樂二店在試賣期間只有販賣鬆餅&朝食...
但最近解除試賣期所以也開始販賣起在一店很夯的漢堡系列!
  
看完上面的照片後大家有沒有發現到一件很古怪的事...
為什麼田樂一樓沒有大門之類的東西...
這樣是要怎麼進去呢(兩手一攤~)

In [4]:
# Take all result
docs = list(elasticsearch.helpers.scan(es, index="pixnet", doc_type='food'))
total_content = [doc['_source'].get('content') for doc in docs]
print(len(total_content))

9597


## Query Emotion

In [5]:
import urllib.request
import json

In [6]:
texts_list = total_content[0].split('。')

In [7]:
query = {"data":[]}
for text in texts_list:
    query["data"].append({"message":text})
query

{'data': [{'message': '說起最近高雄地區新開火紅的早午餐店，這間沐濛一定榜上有名，因為餐點份量大、價格實惠，十分受到好評'},
  {'message': '之前有在一些文章提過，高雄知名的早午餐店我也不是不想吃，只是一週唯一只有禮拜天可以睡到自然醒的我，每次睡醒加上懶懶的東摸西摸一下，可以出門的時間都已經接近中午了，而且也沒動力大老遠跑去一些大排長龍的名店朝聖，對我來說週末就是要懶散到底，別說還要訂位趕時間之類的，饒了我吧！\xa0所以週末比較常就近在鳳山隨便吃一吃，不然就是去我最愛的故事鍋物館吃涮涮鍋'},
  {'message': '這回發現沐濛離鳳山不遠ㄟ，其實跟常跑的文山特區距離差不多而已，好吧，不吃則已，要吃我就來衝第一吧！就這樣，週末的營業時間前，我們在店門口出現囉！（一定要這麼極端嗎？\xa0）時間到，沐濛準時開店囉～！上頭招牌好大片的木板上，只有小小的店名，很特別的設計，店外地板則是黑色鐵板，偏向工業風，黑色窗框外觀沉穩又時尚'},
  {'message': '裡面座位不多，還好有跑第一名，不然就沒辦法拍到這畫面啦！這家店最讓我喜歡的，是磚牆上這些畫框和鏡子，畫框裡有的沒有畫，選擇留白，鏡子裡的倒影，真真假假，虛幻難辨'},
  {'message': '\xa0因為是第一組到的客人，第一位店員對我們十分禮貌殷勤，馬上送上菜單～菜單設計的質感不錯，特別訂製的硬皮'},
  {'message': '以下菜單是從 Moment 沐濛 的官方粉絲團借圖來的～（※後來寫文的時候發現，我們現場看到的菜單品項，和臉書上菜單的版本有一點點不一樣，菜色有增減，請以現場店家提供為主，本菜單僅供參考※）點這一頁早午餐組合是最划算的，都有附飲料喝到飽（冰熱紅茶、熱美式咖啡、經典早餐茶）'},
  {'message': '江湖上（？）傳言捲餅很好吃，但我們還沒吃過'},
  {'message': '這一頁蒜辣炒小管很吸引我，但是他們家餐點份量頗大，通常一個人一個餐就能吃飽，還沒機會點到這邊來'},
  {'message': '義大利麵最貴不超過＄200，只是選擇不多就是'},
  {'message': '吸引我們來的重要理由之一，就是有美式漢堡'},
  {'message': '兒童餐，就是份量小一點，還有用可愛的卡通餐具'},
  

In [8]:
header={'Content-Type': 'application/json'}
req = urllib.request.Request(url='http://192.168.2.100:5678/chuck/couple_all', headers=header, method='POST')
query_str = json.dumps(query)


In [9]:
res = urllib.request.urlopen(req, query_str.encode())
res_json = json.loads(res.read().decode())

In [10]:
emotion_dict = {'wow':0,'love':0,'haha':0,'sad':0,'angry':0}
for sentence_res in res_json['data']:
    if sentence_res['ambiguous']!= True:
        if sentence_res['emotion1'] == 'angry' and sentence_res['emotion2'] == 'haha': continue
        emotion_dict[sentence_res['emotion1']] = emotion_dict[sentence_res['emotion1']] +1
emotion_dict

{'angry': 5, 'haha': 8, 'love': 2, 'sad': 3, 'wow': 9}

In [11]:
res_json

{'data': [{'ambiguous': False,
   'emotion1': 'wow',
   'emotion2': 'sad',
   'location': 'NA',
   'message': '說起最近高雄地區新開火紅的早午餐店，這間沐濛一定榜上有名，因為餐點份量大、價格實惠，十分受到好評'},
  {'ambiguous': False,
   'emotion1': 'haha',
   'emotion2': 'angry',
   'location': 'NA',
   'message': '裡面座位不多，還好有跑第一名，不然就沒辦法拍到這畫面啦！這家店最讓我喜歡的，是磚牆上這些畫框和鏡子，畫框裡有的沒有畫，選擇留白，鏡子裡的倒影，真真假假，虛幻難辨'},
  {'ambiguous': False,
   'emotion1': 'haha',
   'emotion2': 'wow',
   'location': 'NA',
   'message': '\xa0因為是第一組到的客人，第一位店員對我們十分禮貌殷勤，馬上送上菜單～菜單設計的質感不錯，特別訂製的硬皮'},
  {'ambiguous': False,
   'emotion1': 'angry',
   'emotion2': 'haha',
   'location': 'NA',
   'message': '以下菜單是從 Moment 沐濛 的官方粉絲團借圖來的～（※後來寫文的時候發現，我們現場看到的菜單品項，和臉書上菜單的版本有一點點不一樣，菜色有增減，請以現場店家提供為主，本菜單僅供參考※）點這一頁早午餐組合是最划算的，都有附飲料喝到飽（冰熱紅茶、熱美式咖啡、經典早餐茶）'},
  {'ambiguous': False,
   'emotion1': 'love',
   'emotion2': 'wow',
   'location': 'NA',
   'message': '江湖上（？）傳言捲餅很好吃，但我們還沒吃過'},
  {'ambiguous': False,
   'emotion1': 'angry',
   'emotion2': 'sad',
   'location': 'NA',
   'message': '這一頁蒜辣